# Setup

In [1]:
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

# Add Message History (Memory)

The `RunnableWithMessageHistory` lets us **add message history to certain types of chains**.

It:
- wraps another Runnable,
- manages the chat message history for it.

Specifically, it can be used for any runnable that:
-  **takes as input one of**:
    - a sequence of `BaseMessage`,
    - a `dict` with a key that takes a sequence of `BaseMessage`,
    - a `dict` with a key that takes:
        - the latest message(s) as a string or sequence of `BaseMessage`, and
        - a separate key that takes historical messages.
- **and returns as output one of**:
    - a string that can be treated as the contents of an `AIMessage`,
    - a sequence of `BaseMessage`,
    - a `dict` with a key that contains a sequence of `BaseMessage`.

Let's take a look at some examples to see how it works.

First, we construct a runnable which, here:
- accepts a `dict` as input,
- returns a message as output.

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI

In [7]:
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You're an assistant who's good at {ability}. Respond in 20 words or fewer."
    ),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])
runnable = prompt | model

> **API Reference**:
> - [**ChatPromptTemplate**](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html)
> - [**MessagesPlaceholder**](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.MessagesPlaceholder.html)
> - [**ChatOpenAI**](https://api.python.langchain.com/en/latest/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html)

To manage the message history, we will need:
1. this runnable,
2. a callable that returns an instance of `BaseChatMessageHistory`.

You can check out the [**memory integrations**](https://integrations.langchain.com/memory) page for implementations of chat message histories using Redis and other providers.

Here, we will demonstrate using an **in-memory** `ChatMessageHistory` as well as more persistent storage using `RedisChatMessageHistory`.